In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from transformers import BertTokenizer
import pandas as pd
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/bertmodel/new')
from bert import minBert
from bert import EncoderLayer, SelfAttention, FeedForward, BaseAttention
from positional_embedding import PositionalEmbedding
sys.path.append('/content/drive/MyDrive/Colab Notebooks/bertmodel/contrastive_learning')
from contrastive_learning import UnsupervisedSimCSE

In [3]:
mlm_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bertmodel/bookcorpus.csv')
mlm_data = mlm_data.head(500000)

In [4]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
min_bert_layer = minBert(
    name = 'minbert',
    num_layers=4,  # Số lượng lớp encoder
    d_model=256,   # Kích thước vector ẩn
    num_heads=8,   # Số lượng head trong multi-head attention
    dff=1024,      # Số lượng neuron trong feed-forward network
    vocab_size=tokenizer.vocab_size,  # Kích thước từ vựng
    dropout_rate=0.1
)

In [5]:
min_bert_model = tf.keras.models.load_model(
    '/content/drive/MyDrive/Colab Notebooks/bertmodel/min_bert_layer.keras',
    custom_objects={
        "minBert": minBert,
        "EncoderLayer": EncoderLayer,
        "SelfAttention": SelfAttention,
        "FeedForward": FeedForward,
        "BaseAttention" : BaseAttention,
        "PositionalEmbedding" : PositionalEmbedding
    }
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ke

In [6]:
inputs = tokenizer(mlm_data['text'].values.tolist(), return_tensors="tf", add_special_tokens=True, padding=True, truncation=True, max_length=256)

# Tạo tf.data.Dataset từ các tensor trong inputs
mlm_ds = tf.data.Dataset.from_tensor_slices((
    {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "token_type_ids": inputs["token_type_ids"]}
)).batch(64)

In [8]:
simcse_model = UnsupervisedSimCSE(min_bert_model)

simcse_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5))

simcse_model.fit(mlm_ds, epochs=3, batch_size=64)

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ke

7813/7813 ━━━━━━━━━━━━━━━━━━━━ 1339s 168ms/step - loss: 0.0042
Epoch 2/3
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 1320s 167ms/step - loss: 0.0033
Epoch 3/3
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 1304s 167ms/step - loss: 0.0033


In [10]:
min_bert_model.save("min_bert_layer_after_unsupervised_training.keras")